<a href="https://colab.research.google.com/github/GaboV3/yolo_class/blob/main/yolodetec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Instalação das Dependências

Esta célula instala todas as bibliotecas necessárias para o projeto. A `ultralytics` é a biblioteca principal para o uso do YOLO, enquanto as outras são para manipulação de dados geoespaciais e processamento de imagem.

In [ ]:
# Bibliotecas essenciais para o projeto.
!pip install git+https://github.com/ultralytics/ultralytics.git opencv-python-headless matplotlib numpy pyyaml rasterio geopandas shapely tqdm kagglehub -q

## 2. Configuração do Ambiente e Download do Dataset

Para acessar o dataset do Kaggle, é necessário autenticar sua conta. Esta célula irá solicitar o upload do seu arquivo `kaggle.json`, que contém suas credenciais da API do Kaggle. Após a autenticação, o dataset será baixado e o arquivo de configuração `data.yaml` será ajustado para funcionar corretamente no ambiente do Colab.

**Para alterar o dataset, modifique a variável `KAGGLE_DATASET_PATH` nesta célula.**

In [ ]:
import os
from google.colab import files
import kagglehub
import glob
import yaml

# --- PARÂMETRO MODIFICÁVEL ---
# Altere esta variável para o dataset do Kaggle que deseja usar.
KAGGLE_DATASET_PATH = "isabinimam/vehicle-detection-yolov11"
# -----------------------------

def setup_kaggle_and_download_dataset(dataset_path):
    """
    Gerencia o upload do kaggle.json, download e preparação do dataset,
    corrigindo os caminhos no arquivo data.yaml para serem absolutos.
    """
    print("Por favor, faça o upload do seu arquivo 'kaggle.json'")
    try:
        uploaded = files.upload()
        if 'kaggle.json' not in uploaded:
            print("\nUpload cancelado ou arquivo 'kaggle.json' não selecionado.")
            return None
    except Exception as e:
        print(f"\nOcorreu um erro durante o upload: {e}")
        return None

    # Configura as credenciais do Kaggle
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("\nCredenciais do Kaggle configuradas com sucesso!")

    # Baixa e descompacta o dataset
    print(f"Baixando o dataset '{dataset_path}' do Kaggle...")
    try:
        download_path = kagglehub.dataset_download(dataset_path)
        print(f"Dataset baixado e extraído para: {download_path}")
    except Exception as e:
        print(f"Falha ao baixar o dataset. Verifique o caminho do dataset ou suas credenciais. Erro: {e}")
        return None

    # Procura pelo arquivo data.yaml no diretório do dataset
    try:
        original_yaml_path = glob.glob(f'{download_path}/**/data.yaml', recursive=True)[0]
        # Get the root directory of the dataset content (where train/val/test folders are)
        dataset_content_root = os.path.dirname(original_yaml_path) # Corrected: Set to the directory containing train/val/test
        print(f"Arquivo 'data.yaml' original found at: {original_yaml_path}")
        print(f"Dataset content root identified as: {dataset_content_root}")
    except IndexError:
        print("ERRO: Não foi possível encontrar o arquivo 'data.yaml' no dataset baixado.")
        return None

    # Carrega o arquivo de configuração original
    with open(original_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)

    # CORREÇÃO: Define o caminho raiz e os caminhos relativos para train/val/test
    print("Corrigindo os caminhos do dataset...")

    # Set the 'path' key to the directory containing train/val/test
    data_config['path'] = dataset_content_root
    print(f"  - Chave 'path' definida para o caminho raiz do conteúdo do dataset: {dataset_content_root}")

    # Set train/val/test paths relative to the new 'path'
    for key in ['train', 'val', 'test']:
        if key in data_config and data_config[key]:
            # Assuming the original paths in data.yaml were relative to the original yaml location
            # We need to find the path relative to the new dataset_content_root
            original_relative_path = os.path.relpath(os.path.join(os.path.dirname(original_yaml_path), data_config[key]), dataset_content_root)
            data_config[key] = original_relative_path
            print(f"  - Caminho '{key}' atualizado para o caminho relativo: {original_relative_path}")


    # Salva o arquivo de configuração corrigido em um diretório gravável
    corrected_data_yaml_path = '/content/corrected_data.yaml'
    with open(corrected_data_yaml_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False, sort_keys=False)

    print(f"\nArquivo 'data.yaml' corrigido e salvo em: {corrected_data_yaml_path}")
    return corrected_data_yaml_path

# Executa a função de configuração
CORRECTED_DATA_YAML_PATH = setup_kaggle_and_download_dataset(KAGGLE_DATASET_PATH)

Por favor, faça o upload do seu arquivo 'kaggle.json'


Saving kaggle.json to kaggle.json

Credenciais do Kaggle configuradas com sucesso!
Baixando o dataset 'isabinimam/vehicle-detection-yolov11' do Kaggle...
Dataset baixado e extraído para: /kaggle/input/vehicle-detection-yolov11
Arquivo 'data.yaml' original found at: /kaggle/input/vehicle-detection-yolov11/dataset/data.yaml
Dataset content root identified as: /kaggle/input/vehicle-detection-yolov11/dataset
Corrigindo os caminhos do dataset...
  - Chave 'path' definida para o caminho raiz do conteúdo do dataset: /kaggle/input/vehicle-detection-yolov11/dataset
  - Caminho 'train' atualizado para o caminho relativo: ../train/images
  - Caminho 'val' atualizado para o caminho relativo: ../valid/images
  - Caminho 'test' atualizado para o caminho relativo: ../test/images

Arquivo 'data.yaml' corrigido e salvo em: /content/corrected_data.yaml


## 3. Treinamento do Modelo YOLO

Com o dataset pronto, esta célula inicia o processo de treinamento. Os resultados serão salvos no seu Google Drive.

**Para alterar a versão do modelo YOLO (ex: 'yolo11s.pt', 'yolo11m.pt'), modifique a variável `YOLO_MODEL_VERSION`.**

In [ ]:
from ultralytics import YOLO
from google.colab import drive
import os

# --- PARÂMETRO MODIFICÁVEL ---
# Escolha a versão do modelo YOLO que deseja treinar.
# Opções comuns: 'yolo11n.pt' (nano), 'yolo11s.pt' (small), 'yolo11m.pt' (medium)
YOLO_MODEL_VERSION = 'yolo11n.pt'
# -----------------------------

def train_yolo_model(data_yaml_path, model_version):
    """Treina o modelo YOLO com o dataset e a versão especificada."""
    if not data_yaml_path or not os.path.exists(data_yaml_path):
        # Mensagem de erro corrigida para apontar para a célula correta
        print("Caminho do dataset (corrected_data.yaml) não foi definido ou não existe. Execute a Célula 1 primeiro.")
        return None

    # Monta o Google Drive para salvar os resultados
    try:
        drive.mount('/content/drive', force_remount=True)
        results_dir = '/content/drive/MyDrive/yolo/YOLO_Detection_Results'
        os.makedirs(results_dir, exist_ok=True)
    except Exception as e:
        print(f"Não foi possível montar o Google Drive ou criar o diretório. Erro: {e}")
        return None

    # Carrega o modelo pré-treinado
    model = YOLO(model_version)

    # Inicia o treinamento
    print(f"\nIniciando o treinamento do modelo {model_version}...")
    results = model.train(
        data=data_yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        patience=10,
        name=f'{os.path.splitext(model_version)[0]}_detection', # Nome do projeto para salvar os resultados
        project=results_dir,
        exist_ok=True # Permite reutilizar um diretório de projeto se já existir
    )
    print(f"\nTreinamento concluído! Resultados salvos em: {results.save_dir}")

    # Retorna o caminho para os melhores pesos do modelo treinado
    best_model_path = os.path.join(results.save_dir, 'weights/best.pt')
    if os.path.exists(best_model_path):
        print(f"Modelo treinado (melhores pesos) salvo em: {best_model_path}")
        return best_model_path
    else:
        print("Não foi possível encontrar o arquivo 'best.pt' dos pesos do modelo.")
        return None

# Executa o treinamento somente se a Célula 1 foi bem-sucedida
if 'CORRECTED_DATA_YAML_PATH' in locals() and CORRECTED_DATA_YAML_PATH:
    MODELO_TREINADO_PATH = train_yolo_model(CORRECTED_DATA_YAML_PATH, YOLO_MODEL_VERSION)
else:
    print("\nTreinamento não iniciado porque a preparação do dataset na Célula 1 falhou.")

Mounted at /content/drive

Iniciando o treinamento do modelo yolo11n.pt...
Ultralytics 8.3.173 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/corrected_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_detection_run, nbs=64, nms=False, opset=None, optimize=False, optimizer

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,620 parameters, 2,590,604 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.1±0.2 ms, read: 30.5±12.0 MB/s, size: 139.2 KB)


train: Scanning /kaggle/input/vehicle-detection-yolov11/dataset/train/labels... 4176 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4176/4176 [00:22<00:00, 187.64it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/vehicle-detection-yolov11/dataset/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.9±1.1 ms, read: 30.4±19.0 MB/s, size: 130.1 KB)


...

Plotting labels to /content/drive/MyDrive/yolo/YOLO_Detection_Results/yolo11n_detection_run/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/yolo/YOLO_Detection_Results/yolo11n_detection_run
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.36G      1.534       1.94      1.273        119        640: 100%|██████████| 261/261 [01:26<00:00,  3.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  2.10it/s]


                   all        389       2543      0.701      0.593      0.691      0.466

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50      2.75G      1.503      1.389       1.25        147        640:   6%|▌         | 16/261 [00:04<01:05,  3.75it/s]
Exception in thread Thread-21 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/reductions.py", 

KeyboardInterrupt: 

## 4. Detecção em GeoTIFF e Exportação para Shapefile

Esta célula contém a função para realizar a detecção em uma imagem GeoTIFF. Ela processa a imagem em blocos (tiles), aplica o modelo treinado e converte as detecções em polígonos georreferenciados, salvando o resultado em um arquivo Shapefile.

In [ ]:
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from tqdm.notebook import tqdm
import cv2
from ultralytics import YOLO

def detect_and_export_to_shapefile(
    geotiff_path,
    model_path,
    output_shapefile_path,
    tile_size=640,
    overlap=0.2,
    conf_threshold=0.4
):
    """Detecta objetos em um GeoTIFF e exporta os resultados para um Shapefile."""
    if not os.path.exists(geotiff_path):
        print(f"ERRO: O arquivo GeoTIFF não foi encontrado em '{geotiff_path}'")
        return

    if not os.path.exists(model_path):
        print(f"ERRO: O arquivo do modelo treinado não foi encontrado em '{model_path}'")
        return

    print("Carregando o modelo treinado...")
    model = YOLO(model_path)
    class_names = model.names

    print(f"Abrindo o arquivo GeoTIFF: {geotiff_path}")
    with rasterio.open(geotiff_path) as src:
        geotiff_transform = src.transform
        geotiff_crs = src.crs
        h, w = src.height, src.width

        all_polygons, all_confidences, all_class_ids, all_class_names = [], [], [], []
        stride = int(tile_size * (1 - overlap))

        for y in tqdm(range(0, h, stride), desc="Progresso Vertical"):
            for x in tqdm(range(0, w, stride), desc="Progresso Horizontal", leave=False):
                window = rasterio.windows.Window(x, y, tile_size, tile_size)
                tile_data = src.read(window=window)[:3, :, :]

                if tile_data.shape[1] == 0 or tile_data.shape[2] == 0:
                    continue

                tile_data = np.transpose(tile_data, (1, 2, 0))
                if tile_data.dtype != np.uint8 or tile_data.max() > 255:
                     tile_8bit = cv2.normalize(tile_data, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
                     if tile_8bit.shape[-1] == 3:
                         tile_8bit = cv2.cvtColor(tile_8bit, cv2.COLOR_RGB2BGR)
                else:
                    tile_8bit = tile_data
                    if tile_8bit.shape[-1] == 3:
                         tile_8bit = cv2.cvtColor(tile_8bit, cv2.COLOR_RGB2BGR)

                results = model(tile_8bit, conf=conf_threshold, verbose=False)

                for result in results:
                    if result.boxes: # Verifica se há caixas detectadas
                        for box in result.boxes.xyxy.cpu().numpy():
                            px1, py1, px2, py2 = box
                            # Transforma as coordenadas de pixel de volta para coordenadas geográficas
                            top_left_geo = rasterio.transform.xy(geotiff_transform, y + py1, x + px1)
                            bottom_right_geo = rasterio.transform.xy(geotiff_transform, y + py2, x + px2)

                            all_polygons.append(Polygon([
                                top_left_geo, (bottom_right_geo[0], top_left_geo[1]),
                                bottom_right_geo, (top_left_geo[0], bottom_right_geo[1])
                            ]))
                        if result.boxes.conf is not None:
                            for conf in result.boxes.conf.cpu().numpy():
                                all_confidences.append(float(conf))
                        if result.boxes.cls is not None:
                             for cls in result.boxes.cls.cpu().numpy():
                                all_class_ids.append(int(cls))
                                all_class_names.append(class_names[int(cls)])

    if not all_polygons:
        print("Nenhum objeto detectado.")
        return

    print(f"\nTotal de {len(all_polygons)} detecções encontradas. Criando GeoDataFrame...")
    min_len = min(len(all_polygons), len(all_confidences), len(all_class_ids), len(all_class_names))
    if not (len(all_polygons) == len(all_confidences) == len(all_class_ids) == len(all_class_names)):
         print(f"Aviso: Incompatibilidade no número de polígonos, confianças, class_ids ou class_names. Truncando para o comprimento mínimo ({min_len}).")
         all_polygons = all_polygons[:min_len]
         all_confidences = all_confidences[:min_len]
         all_class_ids = all_class_ids[:min_len]
         all_class_names = all_class_names[:min_len]

    gdf = gpd.GeoDataFrame(
        {'confidence': all_confidences, 'class_id': all_class_ids, 'class_name': all_class_names},
        geometry=all_polygons, crs=geotiff_crs
    )

    print(f"Salvando {len(gdf)} detecções no Shapefile: {output_shapefile_path}")
    gdf.to_file(output_shapefile_path, driver='ESRI Shapefile')
    print("Processo concluído com sucesso!")

# --- Execução da Detecção ---
# Usa o caminho hardcoded
MODELO_TREINADO_PATH_HARDCODED = "/content/drive/MyDrive/yolo/YOLO_Detection_Results/yolo11n_detection_run/weights/best.pt"

# Verifica se o caminho hardcoded do modelo existe
if os.path.exists(MODELO_TREINADO_PATH_HARDCODED):
    # --- PARÂMETROS MODIFICÁVEIS ---
    GEOTIFF_ENTRADA_PATH = "/content/drive/MyDrive/yolo/orthophoto.tif"
    SHAPEFILE_SAIDA_PATH = "/content/drive/MyDrive/yolo/detec.shp"
    # -----------------------------

    os.makedirs(os.path.dirname(SHAPEFILE_SAIDA_PATH), exist_ok=True)

    detect_and_export_to_shapefile(
        geotiff_path=GEOTIFF_ENTRADA_PATH,
        model_path=MODELO_TREINADO_PATH_HARDCODED,
        output_shapefile_path=SHAPEFILE_SAIDA_PATH
    )
else:
    print(f"\nO arquivo do modelo treinado não foi encontrado em: {MODELO_TREINADO_PATH_HARDCODED}. Certifique-se de que o treinamento foi concluído com sucesso e o arquivo existe.")

Carregando o modelo treinado...
Abrindo o arquivo GeoTIFF: /content/drive/MyDrive/yolo/orthophoto.tif


Progresso Vertical:   0%|          | 0/22 [00:00<?, ?it/s]

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]

...

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]

Progresso Horizontal:   0%|          | 0/28 [00:00<?, ?it/s]


Total de 160 detecções encontradas. Criando GeoDataFrame...
Salvando 160 detecções no Shapefile: /content/drive/MyDrive/yolo/detec.shp
Processo concluído com sucesso!
